# 감성 분석 모델 학습 및 추론

### 1. 데이터 로드

In [23]:
import pandas as pd

# 데이터셋 URL
train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"

# read_csv 함수를 사용하되, 구분자(sep)를 탭(\t)으로 지정
train_df = pd.read_csv(train_url, sep='\t')
test_df = pd.read_csv(test_url, sep='\t')

# 데이터 상위 5개 확인
display(train_df.head())
display(test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### 2. 데이터 전처리

In [36]:
# 결측치 확인 및 처리
train_df.dropna(subset=['document'], inplace=True)
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.dropna(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

train_df.shape, test_df.shape

((143682, 3), (48418, 3))

In [25]:
# 한글이 아닌 문자 제거
import re

train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', x))
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', x))
print(train_df.head())
print(test_df.head())

         id                                           document  label
0   9976970                                  아 더빙 진짜 짜증나네요 목소리      0
1   3819312                         흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                          교도소 이야기구먼 솔직히 재미는 없다평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...      1
        id                                   document  label
0  6270596                                        굳 ㅋ      1
1  9274899                                                 0
2  8544678           뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아      0
3  6825595                  지루하지는 않은데 완전 막장임 돈주고 보기에는      0
4  6723715  만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠      0


In [ ]:
# 불용어 제거 및 토큰화
from konlpy.tag import Okt

okt = Okt()

# 불용어(Stopwords) 리스트 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

# 토큰화 함수 정의
def tokenize(text):
    tokens = okt.morphs(text, stem=True)  # 형태소 단위로 토큰화
    return [word for word in tokens if word not in stopwords]   # 불용어 제거 

# 토큰화 & 불용어 제거 
X_train_tokenized = [tokenize(doc) for doc in train_df['document']]
X_test_tokenized = [tokenize(doc) for doc in test_df['document']]

In [38]:
X_train_tokenized

[['아더', '빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '익살스럽다',
  '연기',
  '돋보이다',
  '영화',
  '스파이더맨',
  '에서',
  '늙다',
  '보이다',
  '하다',
  '크다',
  '스틴던스트',
  '너무나도',
  '이쁘다',
  '보이다'],
 ['막',
  '걸음',
  '마',
  '떼다',
  '초등학교',
  '학년',
  '생인',
  '살다',
  '영화',
  'ㅋㅋㅋ',
  '별',
  '반개',
  '아깝다',
  '움'],
 ['원작', '긴장감', '을제', '대로', '살리다', '하다'],
 ['별',
  '반개',
  '아깝다',
  '다욕',
  '나오다',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '몇년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '드라마',
  '가족',
  '없다',
  '연기',
  '못',
  '사람',
  '만',
  '모',
  '엿'],
 ['액션', '없다', '재미있다', '몇', '안되다', '영화'],
 ['왜케', '평점', '낮다', '꽤볼', '만', '데', '헐리우드', '식', '화려하다', '너무', '길들이다'],
 ['걍', '인피니트', '짱', '이다', '진짜', '짱', '이다'],
 ['볼때',
  '마다',
  '눈물나다',
  '죽다',
  '년대',
  '향수',
  '자극',
  '허진호',
  '감성',
  '절제

In [26]:
from collections import Counter

word_counts = Counter(word for words in X_train_tokenized for word in words)
vocab_sorted = sorted(word_counts, key=word_counts.get, reverse=True)

# 단어에 정수 인덱스 부여
word_to_int = {word: i + 2 for i, word in enumerate(vocab_sorted)}
word_to_int['<PAD>'] = 0  # 패딩 토큰
word_to_int['<UNK>'] = 1   # 알 수 없는 단어 토큰 (Unknown)
VOCAB_SIZE = len(word_to_int)

In [39]:
word_to_int

{'영화': 2,
 '보다': 3,
 '하다': 4,
 '없다': 5,
 '이다': 6,
 '좋다': 7,
 '너무': 8,
 '정말': 9,
 '재밌다': 10,
 '적': 11,
 '되다': 12,
 '같다': 13,
 '있다': 14,
 '만': 15,
 '진짜': 16,
 '않다': 17,
 '로': 18,
 '으로': 19,
 '아니다': 20,
 '에서': 21,
 '내': 22,
 '평점': 23,
 '그': 24,
 '최고': 25,
 '나': 26,
 '연기': 27,
 '생각': 28,
 '스토리': 29,
 '점': 30,
 '이영화': 31,
 '드라마': 32,
 '감동': 33,
 '사람': 34,
 '만들다': 35,
 '요': 36,
 '나오다': 37,
 '아깝다': 38,
 '이런': 39,
 'ㅋㅋ': 40,
 '보고': 41,
 '재미있다': 42,
 '배우': 43,
 '왜': 44,
 '감독': 45,
 '때': 46,
 '들다': 47,
 '하고': 48,
 '그냥': 49,
 '아': 50,
 '재미없다': 51,
 '시간': 52,
 '까지': 53,
 '못': 54,
 '중': 55,
 '것': 56,
 '지루하다': 57,
 '뭐': 58,
 '가다': 59,
 '재미': 60,
 '쓰레기': 61,
 '말': 62,
 '면': 63,
 '주다': 64,
 '본': 65,
 '모르다': 66,
 '작품': 67,
 '알다': 68,
 '더': 69,
 '오다': 70,
 '거': 71,
 '좀': 72,
 '자다': 73,
 '그렇다': 74,
 '사랑': 75,
 '마지막': 76,
 '저': 77,
 '대': 78,
 '정도': 79,
 '화': 80,
 'ㅠㅠ': 81,
 '이나': 82,
 '많다': 83,
 '완전': 84,
 'ㅋㅋㅋ': 85,
 '처음': 86,
 '라': 87,
 'ㅋ': 88,
 '안되다': 89,
 '개': 90,
 '액션': 91,
 '이렇게': 92,
 '주인공': 93,
 

In [27]:
# 정수 인코딩

X_train_encoded = [[word_to_int.get(word, 1) for word in words] for words in X_train_tokenized]
X_test_encoded = [[word_to_int.get(word, 1) for word in words] for words in X_test_tokenized]

In [40]:
X_train_encoded

[[9876, 11444, 16, 245, 658],
 [1059, 465, 41, 602, 2, 404, 1506, 27, 1074, 677, 17],
 [209, 1716, 27027, 566, 3845, 3, 221],
 [6985, 102, 9321, 211, 60, 5, 23, 4086],
 [1127,
  17645,
  24,
  9877,
  27,
  830,
  2,
  2687,
  21,
  1132,
  279,
  4,
  216,
  13764,
  1101,
  238,
  279],
 [984, 6736, 1102, 1390, 1748, 1663, 12515, 241, 2, 85, 125, 1151, 38, 243],
 [215, 302, 2420, 421, 507, 4],
 [125,
  1151,
  38,
  4164,
  37,
  11445,
  15378,
  27,
  1683,
  5546,
  222,
  9,
  1010,
  18,
  536,
  561,
  563,
  523,
  3306,
  8774,
  15,
  1525,
  1525,
  32,
  280,
  5,
  27,
  54,
  34,
  15,
  842,
  1002],
 [91, 5, 42, 445, 89, 2],
 [1767, 23, 196, 5251, 15, 342, 1555, 316, 663, 8, 5909],
 [371, 4992, 155, 6, 16, 155, 6],
 [653, 387, 1156, 200, 376, 2596, 919, 12516, 595, 2570, 907, 6087, 6],
 [2571, 13765, 27028, 6737, 46, 1398, 261, 37, 988, 3929, 27, 980, 54, 4],
 [2540, 1242, 7, 11446, 3426, 989, 3, 3, 941, 7905, 566, 34, 6, 56],
 [869,
  4002,
  4003,
  15,
  16,
  1260,

In [42]:
# 패딩

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 80 

X_train_padded = pad_sequences(X_train_encoded, padding='post', maxlen=max_len, truncating='post')
X_test_padded = pad_sequences(X_test_encoded, padding='post', maxlen=max_len, truncating='post')

X_train_padded

array([[ 9876, 11444,    16, ...,     0,     0,     0],
       [ 1059,   465,    41, ...,     0,     0,     0],
       [  209,  1716, 27027, ...,     0,     0,     0],
       ...,
       [ 1392,    36,  1531, ...,     0,     0,     0],
       [ 4882,  2339,  3034, ...,     0,     0,     0],
       [  349,    80,  1849, ...,     0,     0,     0]])

### 3. 모델 정의 및 생성

In [29]:
import torch
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        final_hidden_state = hidden.squeeze(0)      # 마지막 hidden state 사용 
        out = self.fc(final_hidden_state)
        return self.sigmoid(out)

In [30]:
import torch.optim as optim

# 모델, 손실 함수, 옵티마이저 정의
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 1

lstm_model = SentimentLSTM(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM
)
criterion = nn.BCELoss() # 이진 교차 엔트로피 손실
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

### 4. 모델 학습

In [31]:
# 전처리 데이터를 PyTorch 텐서로 변환
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long)
y_train_tensor = torch.tensor(train_df['label'].values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.long)
y_test_tensor = torch.tensor(test_df['label'].values, dtype=torch.float32)


In [32]:
# DataLoader 생성
batch_size = 128
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [43]:
# 학습 루프
epochs = 50
train_losses, val_losses, train_accs, val_accs = [], [], [], []

early_stopping_patience = 7
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(epochs):

    lstm_model.train() # 학습 모드
    epoch_loss, correct, total = 0, 0, 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = lstm_model(inputs).squeeze()

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        pred = (outputs > 0.5).float()
        correct += (pred == labels).sum().item()
        total += labels.size(0)

    train_loss = epoch_loss / len(train_loader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # 검증 단계 
    lstm_model.eval() 
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = lstm_model(test_inputs).squeeze()
            loss = criterion(test_outputs, test_labels)
            val_loss += loss.item()
            
            predicted = (test_outputs > 0.5).float()
            val_total += test_labels.size(0)
            val_correct += (predicted == test_labels).sum().item()
    
    val_loss = val_loss / len(test_loader)
    val_acc = val_correct / val_total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stopping_patience:
            print('Early Stopping')
            break

Epoch 1/50 | Train Loss: 0.1341, Train Acc: 0.9503, Val Loss: 0.4784, Val Acc: 0.8372
Epoch 2/50 | Train Loss: 0.0965, Train Acc: 0.9660, Val Loss: 0.5674, Val Acc: 0.8360
Epoch 3/50 | Train Loss: 0.0699, Train Acc: 0.9763, Val Loss: 0.6879, Val Acc: 0.8335
Epoch 4/50 | Train Loss: 0.0535, Train Acc: 0.9820, Val Loss: 0.7622, Val Acc: 0.8287
Epoch 5/50 | Train Loss: 0.0413, Train Acc: 0.9861, Val Loss: 0.8765, Val Acc: 0.8317


KeyboardInterrupt: 

### 5. 추론

In [ ]:
import numpy as np

def predict_sentiment(sentence, model, word_to_int, tokenizer_func, max_len):
    model.eval()
    
    # 1. 새로운 문장 전처리
    cleaned_sentence = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '', sentence)
    tokenized_sentence = tokenizer_func(cleaned_sentence)
    encoded_sentence = [word_to_int.get(word, 1) for word in tokenized_sentence]
    
    # 2. 패딩
    padded_sentence = np.zeros((1, max_len))
    pad_len = min(len(encoded_sentence), max_len)
    padded_sentence[0, :pad_len] = encoded_sentence[:pad_len]
    
    # 3. 텐서로 변환
    input_tensor = torch.tensor(padded_sentence, dtype=torch.long)
    
    # 4. 예측
    with torch.no_grad():
        output = model(input_tensor)
        prediction = output.item()
        
        
    # 5. 결과 해석
    if prediction > 0.5:
        return f"긍정 (확률: {prediction*100:.2f}%)"
    else:
        return f"부정 (확률: {(1-prediction)*100:.2f}%)"

# 예측해보기
new_sentence1 = "이 영화 진짜 역대급으로 재밌었어요! 배우들 연기 미쳤다"
new_sentence2 = "시간 아깝다... 스토리도 없고 지루하기만 함"
new_sentence3 = "그냥 평범한 영화인듯"

print(f'"{new_sentence1}" -> 예측: {predict_sentiment(new_sentence1, lstm_model, word_to_int, tokenize, max_len)}')
print(f'"{new_sentence2}" -> 예측: {predict_sentiment(new_sentence2, lstm_model, word_to_int, tokenize, max_len)}')
print(f'"{new_sentence3}" -> 예측: {predict_sentiment(new_sentence3, lstm_model, word_to_int, tokenize, max_len)}')

"이 영화 진짜 역대급으로 재밌었어요! 배우들 연기 미쳤다" -> 예측: 긍정 (확률: 62.84%)
"시간 아깝다... 스토리도 없고 지루하기만 함" -> 예측: 긍정 (확률: 62.84%)
"그냥 평범한 영화인듯" -> 예측: 긍정 (확률: 62.84%)
